# Diseño de algoritmos
## Backtracking 

Esta técnica para resolver problemas de optimización probando las diferentes configuraciones. Vamos a aplicarlo a un problema muy conocido, que es el de las N reinas que consta de acomodar N reinas en un tablero de ajedrez sin que se ataquen entre ellas (usualmente N=9, y en este problema usaremos 4).

In [1]:
import numpy as np


### Condiciones y funciones auxiliares

Primero necesitamos funciones auxiliares que nos van a ayudar en nuestro problema. 

In [51]:
A = np.arange(16).reshape(4,4)

In [52]:
A

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

¿Cómo se obtiene la i-ésima de columna / renglón de una matriz en numpy?

La i-ésima fila:

    A[i] <- fila i

La i-ésima columna:

    A[:,i] <- todas las filas, columna i

In [53]:
A[2]

array([ 8,  9, 10, 11])

In [54]:
A[:,2]

array([ 2,  6, 10, 14])

Las diagonales requieren un _hack_ más específico, ocupando la función `diagonal` que tienen todos los arrays en numpy, vamos a juntar todas las diagonales de nuestra matriz en una lista, primero las inversas y luego las diagonales normales.

La pregunta en GitHub la pueden ver [acá](https://stackoverflow.com/a/6313414)

In [68]:
diagsR=[(i,A[::-1,:].diagonal(i)) for i in range(-A.shape[0]+1,A.shape[0])]
diagsN=[(i,A.diagonal(i)) for i in range(A.shape[1]-1, -A.shape[1],-1)]

print(diagsR)
print(diagsN)

[(-3, array([0])), (-2, array([4, 1])), (-1, array([8, 5, 2])), (0, array([12,  9,  6,  3])), (1, array([13, 10,  7])), (2, array([14, 11])), (3, array([15]))]
[(3, array([3])), (2, array([2, 7])), (1, array([ 1,  6, 11])), (0, array([ 0,  5, 10, 15])), (-1, array([ 4,  9, 14])), (-2, array([ 8, 13])), (-3, array([12]))]


¿Por qué queremos hacer esto? 

Por la naturaleza de las reinas en el ajedrez, imponen una condición muy concreta en el juego: eres susceptible a ser atacado por una reina si estás en su fila, renglón o diagonal (normal o inversa). Ahora, si en hay una reina en una casilla implica que no podremos poner otra reina en su fila, renglón o diagonal.

Esto lo vamos a lograr porque nuestro tablero tendrá puros ceros y codificaremos las reinas con un 1. Entonces si sumamos un renglón, columna o diagonal, la suma de sus entradas no debería ser mayor a 1.

In [71]:
#Así sumamos una fila
print(sum(A[0]))
#Así sumamos una columna
print(sum(A[:,0]))

6
24


In [106]:
tablero = np.zeros((4,4))

#tablero válido
tablero[1,1] = 1
tablero[2,3] = 1
print(tablero)

diagsR=dict([(i,tablero[::-1,:].diagonal(i)) for i in range(-tablero.shape[0]+1,tablero.shape[0])])
diagsN=dict([(i,tablero.diagonal(i)) for i in range(tablero.shape[1]-1, -tablero.shape[1],-1)])


def get_diagonales(c,N=3):
    """
    Funcion que regresa el índice que
    corresponde al a diagonal normal (diagsN)
    y a la diagonal invertida (diagsR) correspondientes
    a la coordenada c -en un tablero de tamaño 3-
    """
    n,m = c
    return m-n, n+m-N



[[0. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 0.]]


In [107]:
N, R = get_diagonales((1,1))
print(N,R)
print(diagsN[N])
print(diagsR[R])

0 -1
[0. 1. 0. 0.]
[0. 1. 0.]


In [112]:
print("Para el punto en (1,1)")
n,m = 1,1
sumaf, sumac = sum(tablero[n]), sum(tablero[:,m])
#obtenemos los indices del diccionario para las diagonales
#del punto (1,1)
N,R = get_diagonales((n,m)) 
suman, sumar = sum(diagsN[N]), sum(diagsR[R])
print("Suma fila {}, columna {}, diagonal {}, diagonal r {}".format(sumaf, sumac, suman, sumar))

print("\nPara el punto (2,3)")
n,m = 2, 3
sumaf, sumac = sum(tablero[n]), sum(tablero[:,m])
#obtenemos los indices del diccionario para las diagonales
#del punto (2,3)
N,R = get_diagonales((n,m)) 
suman, sumar = sum(diagsN[N]), sum(diagsR[R])
print("Suma fila {}, columna {}, diagonal {}, diagonal r {}".format(sumaf, sumac, suman, sumar))


Para el punto en (1,1)
Suma fila 1.0, columna 1.0, diagonal 1.0, diagonal r 1.0

Para el punto (2,3)
Suma fila 1.0, columna 1.0, diagonal 1.0, diagonal r 1.0


In [113]:
#tablero inválido
tablero[1,2]=1
tablero

array([[0., 0., 0., 0.],
       [0., 1., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.]])

In [114]:
print("Para el punto en (1,2)")
n,m = 1,2
sumaf, sumac = sum(tablero[n]), sum(tablero[:,m])
#obtenemos los indices del diccionario para las diagonales
#del punto (1,1)
N,R = get_diagonales((n,m)) 
suman, sumar = sum(diagsN[N]), sum(diagsR[R])
print("Suma fila {}, columna {}, diagonal {}, diagonal r {}".format(sumaf, sumac, suman, sumar))


Para el punto en (1,2)
Suma fila 2.0, columna 1.0, diagonal 2.0, diagonal r 1.0


In [185]:
def es_valida(T,c,N=3):
    """
    Funcion que regresa True si la coordenada c en el
    tablero suma algo mayor a 1
    """
    n,m = c
    N,R = get_diagonales(c,N)

    if(sum(T[n])>0):
        #print(1)
        return False
    elif(sum(T[:,m])>0):
        #print(2)
        return False
    elif(sum(diagsN[N])>0):
        #print(3)
        return False
    elif(sum(diagsR[R])>0):
        #print(4)
        return False
    else:
        return True


### Exploración del árbol de estados

Ahora lo que debemos hacer es explorar los diferentes estados para el juego de las 4-reinas e ir verificando que podamos poner cada una de las 4 reinas sin que se ataquen entre ellas.

Para hacer la exploración debemos hacer lo siguiente. Si el contador indica que estamos en la misma profundidad del árbol (en este caso 4) entonces terminamos. Si no, quiere decir que podemos poner una reina más y verificar si la celda en la que está no sería atacada, en caso de fallar la tenemos que quitar y probar en otra celda.

In [193]:
def prueba(n, contador, tablero):
    N = tablero.shape[0]-1
    if(n==contador):
        return True
    for i in range(n):             #exploramos el tablero
        for j in range(n):
            if( es_valida(tablero, (i,j),N) ):
                tablero[i,j] = 1  #ponemos una reina
                contador += 1
                if(prueba(n,contador, tablero)):
                    return True
                tablero[i,j] = 0  #si no fue valido
                contador -= 1
            
    return False

In [200]:
tablero = np.zeros((4,4))
diagsR=dict([(i,tablero[::-1,:].diagonal(i)) for i in range(-tablero.shape[0]+1,tablero.shape[0])])
diagsN=dict([(i,tablero.diagonal(i)) for i in range(tablero.shape[1]-1, -tablero.shape[1],-1)])


In [201]:
prueba(4,0, tablero)

True

In [202]:
tablero

array([[0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.]])

Lo podemos hacer de 5x5

In [197]:
tablero = np.zeros((5,5))
diagsR=dict([(i,tablero[::-1,:].diagonal(i)) for i in range(-tablero.shape[0]+1,tablero.shape[0])])
diagsN=dict([(i,tablero.diagonal(i)) for i in range(tablero.shape[1]-1, -tablero.shape[1],-1)])


In [198]:
prueba(5,0, tablero)

True

In [199]:
tablero

array([[1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.]])